In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install pandas tqdm sentence-transformers

In [3]:
import pandas as pd
from tqdm import tqdm

dataset_path = "/content/drive/MyDrive/Projetos/recipes_project/recipenlg.csv"

chunksize = 200000
filtered_chunks = []

for chunk in tqdm(pd.read_csv(dataset_path, chunksize=chunksize)):
    # filtrar cada chunk antes de guardar
    filtered = chunk[chunk['source'] == 'Gathered']
    if not filtered.empty:
        filtered_chunks.append(filtered)

# montar o dataset filtrado completo
df_filtered = pd.concat(filtered_chunks, ignore_index=True)
df_filtered.info()
df_filtered.head(3)

12it [00:44,  3.69s/it]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1643098 entries, 0 to 1643097
Data columns (total 7 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   Unnamed: 0   1643098 non-null  int64 
 1   title        1643097 non-null  object
 2   ingredients  1643098 non-null  object
 3   directions   1643098 non-null  object
 4   link         1643098 non-null  object
 5   source       1643098 non-null  object
 6   NER          1643098 non-null  object
dtypes: int64(1), object(6)
memory usage: 87.8+ MB


,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."


In [4]:
# filtrar: manter apenas colunas úteis para o sistema
df_filtered = df_filtered[['title', 'ingredients', 'directions', 'link', 'NER']]

# padronizar nomes das colunas
df_filtered.rename(columns={'directions': 'instructions'}, inplace=True)
df_filtered.rename(columns={'NER': 'ner'}, inplace=True)
df_filtered.dropna(subset=['ingredients'], inplace=True)
df_filtered.shape

(1643098, 5)

In [5]:
import ast

# converter 'ner' (string → lista) e deixá-la em minúsculo
def normalize_ner(value):
    try:
        parsed = ast.literal_eval(value)
        if isinstance(parsed, list):
            return [str(x).lower() for x in parsed]
        else:
            return parsed
    except:
        return value.lower() if isinstance(value, str) else value

df_filtered['ner'] = df_filtered['ner'].apply(normalize_ner)


In [6]:
# reduzir o tamanho do dataset
df_small = df_filtered.sample(n=100000, random_state=42).reset_index(drop=True)


In [8]:
# exportar o dataset reduzido
output_path = "/content/drive/MyDrive/Projetos/recipes_project/recipenlg_reduzido.csv"
df_small.to_csv(output_path, index=False)
print("Arquivo salvo em:")
print(output_path)


Arquivo salvo em:
/content/drive/MyDrive/Projetos/recipes_project/recipenlg_reduzido.csv
